<h2>Taller 10 - Pick and place usando control cartesiano</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, otoño 2023


Desde CoppeliaSim, abrir la escena 'scara_pick_n_place.ttt'


<h3>1. Nos conectamos y enviamos al robot a una posición inicial deseada</h3>

In [10]:
# Cargamos las librerías necesarias
import zmqRemoteApi
import time
from cartesian import delta_q


In [11]:
# creamos un cliente de conexión a CoppeliaSim y obtenemos acceso a sim
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

In [12]:
# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
suctionPad = sim.getObject('/base/suctionPad')

print(j1, j2, j3, suctionPad)

# Enviamos al robot la posición deseada
q1_val = 0 * 3.1416/180
q2_val = 0.2
q3_val = 0 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

19 22 26 31


1

<h3>2. Verificamos que el suctionPad funciona bien</h3>

In [26]:
def setEffector(val):
# acciona el efector final (suctionPad)
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    objectHandle = sim.getObject('/base/suctionPad')
    scriptHandle = sim.getScript(sim.scripttype_childscript,objectHandle)
    res = sim.callScriptFunction('setEffector',scriptHandle,val)
    return res

In [27]:
# verificamos si el Pad se activa y desactiva
setEffector(0)

'suctionPad OFF'

In [28]:
# 1. Nos posicionamos sobre la caja
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
suctionPad = sim.getObject('/base/suctionPad')
q1_val = 45 * 3.1416/180
q2_val = 0.2
q3_val = 45 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

1

In [29]:
# 2, Bajamos a la altura de la caja
q2_val = 0.022
sim.setJointTargetPosition(j2, q2_val)

1

In [30]:
#activamos el actuador
setEffector(1)

'suctionPad ON'

In [31]:
# y subimos
q2_val = 0.2
sim.setJointTargetPosition(j2, q2_val)

1

In [32]:
#desactivamos el actuador
setEffector(0)

'suctionPad OFF'

In [33]:
# recogemos con pausas de tiempo
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(1)
sim.setJointTargetPosition(j2, 0.2)
time.sleep(3)
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(0)
sim.setJointTargetPosition(j2, 0.2)

1

<h3>3. Movemos con control cartesiano </h3>

In [34]:
# 1. posicionamos el robot
q1_inicial = 45 * 3.1416/180
q2_inicial = 0.2
q3_inicial = 45 * 3.1416/180

sim.setJointTargetPosition(j1, q1_inicial)
sim.setJointTargetPosition(j2, q2_inicial)
sim.setJointTargetPosition(j3, q3_inicial)

1

In [38]:
# leemos la posicón actual
import sympy
q1_actual = sim.getJointPosition(j1)
q2_actual = sim.getJointPosition(j2)
q3_actual = sim.getJointPosition(j3)
q_actual = [q1_actual, q2_actual, q3_actual]
print(q_actual)

# definimos el vector de desplazamiento
dx = 0.01
dy = 0
delta_x = [dx, dy, 0]

# calculamos la nueva posición
q_nuevo = sympy.Matrix(q_actual) + delta_q(q_actual, delta_x)
print(q_nuevo)

# enviamos al robot las nuevas posiciones
sim.setJointTargetPosition(j1, float(q_nuevo[0]))
sim.setJointTargetPosition(j2, float(q_nuevo[1]))
sim.setJointTargetPosition(j3, float(q_nuevo[2]))

[0.7853999872111146, 0.19999951134718025, 0.7853880230346877]
Matrix([[0.785400575285904], [0.199999511347180], [0.738438687697777]])


1

In [40]:
# recogemos con pausas de tiempo
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(1)
sim.setJointTargetPosition(j2, 0.2)
time.sleep(3)
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(0)
sim.setJointTargetPosition(j2, 0.2)

1

In [41]:
import time
import serial
import zmq
import numpy as np

# Configuración del puerto serial
serial_port = 'COM3'  # Cambiar al puerto serial correcto
baud_rate = 9600

# Configuración de la comunicación con CoppeliaSim
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")  # Cambiar la dirección IP y el puerto según la configuración de CoppeliaSim

# Configuración de las posiciones de las articulaciones
posicion_articulacion_1 = 0.0
posicion_articulacion_2 = 0.0

# Conexión al puerto serial
arduino = serial.Serial(serial_port, baud_rate, timeout=0.1)
time.sleep(2)  # Esperar para establecer la conexión

# Función para enviar comandos a CoppeliaSim
def enviar_comando(comando):
    socket.send_string(comando)
    respuesta = socket.recv_string()
    return respuesta

# Función para procesar los datos recibidos desde el puerto serial
def procesar_datos(datos):
    # Separar los datos en coordenadas x, y y estado del botón
    x, y, boton = datos.split(',')
    x = float(x.split('=')[1])
    y = float(y.split('=')[1])
    boton = bool(int(boton.split('=')[1]))
    
    # Realizar el procesamiento necesario para controlar las posiciones de las articulaciones
    
    # Ejemplo: Calcular las posiciones de las articulaciones basadas en las coordenadas x e y
    global posicion_articulacion_1, posicion_articulacion_2
    posicion_articulacion_1 = x * 0.1  # Ejemplo de cálculo ficticio
    posicion_articulacion_2 = y * 0.1  # Ejemplo de cálculo ficticio
    
    # Enviar las posiciones de las articulaciones a CoppeliaSim
    comando_posicion_1 = f'set_joint_position(joint1, {posicion_articulacion_1})'
    comando_posicion_2 = f'set_joint_position(joint2, {posicion_articulacion_2})'
    enviar_comando(comando_posicion_1)
    enviar_comando(comando_posicion_2)

# Función para controlar el actuador de la ventosa
def controlar_ventosa(accion):
    comando_ventosa = f'set_ventosa({accion})'
    enviar_comando(comando_ventosa)

# Bucle principal
while True:
    try:
        # Leer los datos desde el puerto serial
        datos = arduino.readline().decode().strip()
        
        if datos:
            # Procesar los datos recibidos
            procesar_datos(datos)
            
            # Controlar el actuador de la ventosa según el estado del botón
            if boton:
                controlar_ventosa('agarrar')
            else:
                controlar_ventosa('soltar')
        
        # Esperar un breve período de tiempo antes de leer nuevamente los datos
        time.sleep(0.1)
        
    except KeyboardInterrupt:
        break

# Cerrar la conexión con el puerto serial y detener la simulación en CoppeliaSim
arduino.close()
enviar_comando('simStopSimulation')


SerialException: could not open port 'COM3': FileNotFoundError(2, 'El sistema no puede encontrar el archivo especificado.', None, 2)